In [1]:
import json, re
import numpy as np
import pandas as pd
from konlpy.tag import Okt

In [2]:
okt = Okt(jvmpath='/Library/Java/JavaVirtualMachines/zulu-11.jdk/Contents/Home/bin/java')

[0.005s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016b1e0000-0x000000016b1ec000).
[0.005s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [3]:
playlist_path = './music/json_data/train.json'
with open(playlist_path, 'r', encoding='UTF8') as f:
    data = json.load(f)
df = pd.json_normalize(data)
df.sort_values(by=['like_cnt','updt_date'], ascending=False, inplace=True)

In [4]:
okt = Okt(jvmpath='/Library/Java/JavaVirtualMachines/zulu-11.jdk/Contents/Home/bin/java')

In [5]:
# plylst_title 전처리
def preprocessing(okt, text: str):
    stopwords = ['노래', '음악', '빌보드', '멜론', '플레이리스트', '모음', '리스트', '제목', '모음집', '플레이', '년대', '그때', '명곡', '뮤직', '정말', '쯔음', '차트', '영화']
    text = re.sub('\([<{^)]>}*\)', repl = '', string = text)
    text = re.compile('[^ ㄱ-ㅣ가-힣+]').sub('', text).strip()
    return [word for [word, pos] in okt.pos(text, norm=True, stem=True) if pos == 'Noun' and 1 < len(word) < 5 and word not in stopwords]

In [6]:
tags_data = []
for idx, row in df.iterrows():
    tags_data.append(row['tags'] + preprocessing(okt, row['plylst_title']))

In [7]:
from gensim.models import FastText
model = FastText(tags_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 1)

In [22]:
model.save('./trained_fasttext.model')

In [26]:
model.wv.similarity('새벽', '슬픔')

0.6808478